## 4. Pytorch Custom Datasets
Previously we used in-built dataset in Pytorch(FashionMNIST) Here we will be using a custom dataset to build a model

### What we're going to cover
We will be applying the same workflow we used in the previous notebooks.But instead we will be using our own dataset of pizza,steak and sushi images.


### 0. Importing Pytorch and seeting up device agnostics

In [1]:
import torch
from torch import nn

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### 1. Get data
The data we are going to use is from dataset called "Food 101 dataset". This contains 1000 images of 101 different kinds of food, totalling 101,100 images .But ,instead of 101 food classes , we're going to start with 3:pizza,steak and sushi. and intead of 1000 images per classes we're going to start with random 10% and increase from there

In [4]:
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
    # Download pizza, steak, sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...") 
        zip_ref.extractall(image_path)

Did not find data\pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


### 2. Become one with the data(data preparation)
Before starting a project or building any kind of model, it's important to know what data you're working with.<br>
In our case, we have images of pizza,steak and sushi in standard image classification format.<br>
Image classification format contains separate classes of images in seperate directories titled with a particular class name



The goal will be to **take this data storage structure and turn it into a dataset usable with Pytorch.**

We can inspect what's in our data directory by writing a small helper function to walk through each of the subdirectories and count the files present.<br>
To do so, we'll use Python's in-built os.walk()

In [5]:
import os
def walk_through_dir(dir_path):
    """
    Walks through dir_path returning it's contents.
    Args:
      dir_paths(str or pathlib.Path) : target directory
    Returns:
      A print out of:
      number of subdirectories in dir_path
      number of images(files) in each subdirectory
      name of each subdirectory
    """
    for dirpath,dirnames,filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

In [6]:
walk_through_dir(image_path)

There are 2 directories and 0 images in 'data\pizza_steak_sushi'
There are 3 directories and 0 images in 'data\pizza_steak_sushi\test'
There are 0 directories and 25 images in 'data\pizza_steak_sushi\test\pizza'
There are 0 directories and 19 images in 'data\pizza_steak_sushi\test\steak'
There are 0 directories and 31 images in 'data\pizza_steak_sushi\test\sushi'
There are 3 directories and 0 images in 'data\pizza_steak_sushi\train'
There are 0 directories and 78 images in 'data\pizza_steak_sushi\train\pizza'
There are 0 directories and 75 images in 'data\pizza_steak_sushi\train\steak'
There are 0 directories and 72 images in 'data\pizza_steak_sushi\train\sushi'


In [ ]:
# let's setup our training and testing paths.
